In [2]:
import pandas as pd

Checking an example csv for heart rate


In [3]:
df_ex_hr=pd.read_csv('datasets/COVID-19-Wearables/A0KX894_hr.csv')
df_ex_hr.head(10)

,user,datetime,heartrate
0,A0KX894,2026-07-22 00:00:04,78
1,A0KX894,2026-07-22 00:00:09,77
2,A0KX894,2026-07-22 00:00:14,76
3,A0KX894,2026-07-22 00:00:19,75
4,A0KX894,2026-07-22 00:00:24,74
5,A0KX894,2026-07-22 00:00:29,75
6,A0KX894,2026-07-22 00:00:34,74
7,A0KX894,2026-07-22 00:00:49,74
8,A0KX894,2026-07-22 00:00:54,77
9,A0KX894,2026-07-22 00:00:59,76


In [4]:
df_ex_hr.describe()

,heartrate
count,879561.000000
mean,76.706986
std,14.771697
min,47.000000
25%,66.000000
50%,75.000000
75%,85.000000
max,181.000000


In [5]:
df_ex_hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879561 entries, 0 to 879560
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user       879561 non-null  object
 1   datetime   879561 non-null  object
 2   heartrate  879561 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 20.1+ MB


In [6]:
df_ex_hr['datetime'].describe()

count                  879561
unique                 879561
top       2026-07-22 00:00:04
freq                        1
Name: datetime, dtype: object

Max heart rate is normally 220 - 20, as the data recorded complies with the standards of Stanford universities, the participants should be around or superior to 20. lets do a quick check on one of the recorded data


In [7]:
nan_entries = df_ex_hr.isna().sum()
illogical_entries = df_ex_hr[(df_ex_hr['heartrate'] < 30)| (df_ex_hr['heartrate'] > 200)].sum()
print('Number of NaN entries: ', nan_entries)
print('Number of illogical entries: ', illogical_entries)

Number of NaN entries:  user         0
datetime     0
heartrate    0
dtype: int64
Number of illogical entries:  user         0.0
datetime     0.0
heartrate    0.0
dtype: float64


In [8]:
df_ex_steps=pd.read_csv('datasets/COVID-19-Wearables/A0KX894_hr.csv')
df_ex_steps.head(10)

,user,datetime,heartrate
0,A0KX894,2026-07-22 00:00:04,78
1,A0KX894,2026-07-22 00:00:09,77
2,A0KX894,2026-07-22 00:00:14,76
3,A0KX894,2026-07-22 00:00:19,75
4,A0KX894,2026-07-22 00:00:24,74
5,A0KX894,2026-07-22 00:00:29,75
6,A0KX894,2026-07-22 00:00:34,74
7,A0KX894,2026-07-22 00:00:49,74
8,A0KX894,2026-07-22 00:00:54,77
9,A0KX894,2026-07-22 00:00:59,76


In [9]:
df_ex_steps.describe()

,heartrate
count,879561.000000
mean,76.706986
std,14.771697
min,47.000000
25%,66.000000
50%,75.000000
75%,85.000000
max,181.000000


In [10]:
df_ex_steps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879561 entries, 0 to 879560
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user       879561 non-null  object
 1   datetime   879561 non-null  object
 2   heartrate  879561 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 20.1+ MB


In [11]:
df_ex_steps['datetime'].describe()

count                  879561
unique                 879561
top       2026-07-22 00:00:04
freq                        1
Name: datetime, dtype: object

Lets ittereate over the directories and save a dictionary containing the users name and the corresponding hr and steps

In [12]:
import os
dir_csv = 'datasets/COVID-19-Wearables/'
files = [f for f in os.listdir(dir_csv) if f.endswith('.csv')] # we can straight do it with os.listdir
# noticed there are some files with user name containing_1 or _2, so we need to put that into consideration
#user_name = [f.split('_')[0] if f.count("_")==1 else ""for f in files]
# will deal with it inside the dictionary better
user_name = [f.split('_')[0] for f in files]
mult=0
for i in user_name:
    if(user_name.count(i)) < 1:
        print('missing data for user: ', i)
    if(user_name.count(i)) > 2:
        mult+=1
       
user_name_unique = list(set(user_name))

dict_user_data={}   
for i in user_name_unique:
    dict_user_data[i] = dict()# I am not concatinating strings, there is a user with multi entries
    dict_user_data[i]['hr'] = [f for f in files if f == i+'_hr.csv']
    dict_user_data[i]['steps'] = [f for f in files if f == i+'_steps.csv']
    s=[f for f in files if f == i+'_sleep.csv']
    dict_user_data[i]['sleep'] = s if len(s)!=0 else None
    if files.count(i+'_steps_longterm.csv.csv')>0:
        dict_user_data[i]['steps_longterm']=i+'_steps_longterm.csv'
    if files.count(i+'_hr_longterm.csv.csv')>0:
        dict_user_data[i]['hr_longterm']=i+'_hr_longterm.csv'
    if files.count(i+'_sleep_longterm.csv.csv')>0:
        dict_user_data[i]['sleep_longterm']=i+'_sleep_longterm.csv'

import json
with open('datasets/dict_user_data.json', 'w') as fp:
    json.dump(dict_user_data, fp)
    
# to visualize all the directory structure


In [13]:
merged_df_same_user_dict = {}

for user in dict_user_data.keys():
    if dict_user_data[user]['hr'] != []:
        df_hr = pd.read_csv(dir_csv + dict_user_data[user]['hr'][0])
        df_hr.set_index('datetime', inplace=True)
    if dict_user_data[user]['steps'] != []:
        df_steps=pd.read_csv(dir_csv + dict_user_data[user]['steps'][0])
        df_steps.set_index('datetime', inplace=True)
    merged_df_same_user_dict[user] = df_hr.merge(df_steps, how='outer')


: 

: 